In [96]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sp

from sympy import latex

from IPython.display import display, Math

In [97]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

C_s incomplete & power is incomplete

In [98]:
n_w = None
n_g = 30

N_g = 40 
N_w = 1

D_g = None
D_w = 1.25

P_d = 10
p = None #circular pitch
P_x = None #axial pitch

power = None # Power in HP
life_time = 15000 # in hours

T_o = 100 #input Torque lb-in



idk how to find Face width, so its going right here

In [99]:
F = 0.625

In [100]:
phi_n = 14.5
phi_t = None

## Calculate standard variables

### Calculate pitches and gear diameter

$$p = \frac{\pi}{P_d}=\frac{\pi{}D_G}{N_G}$$
$$P_x=p$$

In [101]:
if p is None and P_d is not None:
    p = np.pi / P_d
    P_x = p
    D_g = N_g / P_d

if p is None and P_d is None:
    p = P_x
    P_d = np.pi / P_x
    D_g = N_g / P_d

if p is not None:
    P_x = p
    P_d = np.pi / p
    D_g = N_g / P_d

if D_g is not None:
    P_d = N_g / D_g
    p = np.pi / P_d
    P_x = p


assert(P_d is not None); assert(p is not None); assert(P_x is not None); assert(D_g is not None)
P_d, p, P_x, D_g

display(Math(f"P_d = {P_d:.2f}"))
display(Math(f"p = {p:.2f}"))
display(Math(f"P_x = {P_x:.2f}"))
display(Math(f"D_g = {D_g:.2f}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### calc L, $\lambda$

$$L = N_WP_x$$

In [102]:
L = N_w * P_x
L

0.3141592653589793

### Compute the lead angle
$$\lambda = \frac{180^\circ{}}{\pi}\cdot{}\arctan{\Big(\frac{L}{\pi{}D_W}\Big)}$$

In [103]:
var_lambda = 180 / np.pi * np.arctan(L / (np.pi * D_w))
var_lambda
display(Math(r"\lambda = {:.2f}^\circ".format(var_lambda)))

<IPython.core.display.Math object>

### copmute normal circular pitch
$$p\cdot{}\cos{\Big(\lambda\cdot{}\frac{\pi}{180}\Big)}$$

In [104]:
p_n = p * np.cos(var_lambda  * np.pi / 180)
p_n
display(Math(r"p_n = {:.2f} \, \text{{rads}}".format(p_n)))

<IPython.core.display.Math object>

### Pitch line speed - UNITS!!!


$$v_{tG} = \frac{\pi{}D_Gn_G}{12}$$

In [105]:
##v_t

v_tG = np.pi * D_g * n_g / 12
v_tG
display(Math(latex(sp.Eq(sp.Symbol('v_tG'), v_tG)) + r" \text{ ft/min}"))

<IPython.core.display.Math object>

### Compute Velcoity ratio and gear ratio

$$\text{VR}=\frac{n_W}{n_g}=\frac{N_g}{N_W}$$

In [106]:

if n_w is not None:
    VR = n_w / n_g
else: 
    VR = N_g / N_w
    n_w = n_g * VR

VR, n_w

(40.0, 1200.0)

In [130]:
m_g = VR

In [108]:
assert(VR == N_g / N_w); assert(VR == n_w / n_g)

### sliding speed
$$v_s = \frac{v_{tG}}{\sin{\lambda}}$$

In [131]:
v_s = v_tG / np.sin(var_lambda * np.pi / 180)
display(Math(latex(sp.Eq(sp.Symbol('v_s'), v_s)) + r" \text{ ft/min}"))

<IPython.core.display.Math object>

### coefficient of friction

In [136]:
mu = None
if v_s == 0:
    mu = 0.15
elif v_s < 10:
    mu = 0.124*np.exp(-0.074 * v_s **(0.645) )
elif v_s >= 10:
    mu = 0.103 * np.exp(-0.11 * v_s **(0.45) ) + 0.012

assert(mu is not None)
display(Math(latex(sp.Eq(sp.Symbol('mu'), mu)) + r" \text{}"))

<IPython.core.display.Math object>

## Compute forces

$$W_{tG} = \frac{T_o}{D_g}$$

In [138]:
assert(T_o is not None)

W_tG = 2 * T_o / D_g
display(Math(latex(sp.Eq(sp.Symbol('W_tG'),W_tG)) + r" \text{ lbf}"))

<IPython.core.display.Math object>

$$W_{xG}=\text{just look in the bible}$$

In [139]:
W_xG = W_tG * ( np.cos(phi_n * np.pi / 180) * np.sin(var_lambda * np.pi / 180) + mu * np.cos(var_lambda * np.pi / 180) ) / ( np.cos(phi_n * np.pi / 180) * np.cos(var_lambda * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) ) 
display(Math(latex(sp.Eq(sp.Symbol('W_xG'),W_xG)) + r" \text{ lbf}"))

<IPython.core.display.Math object>

$$W_{xG}=\text{just look in the bible}$$

In [140]:
W_rG = W_tG * np.sin(phi_n * np.pi / 180) / ( np.cos(var_lambda * np.pi / 180) *np.cos(phi_n * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) )
display(Math(latex(sp.Eq(sp.Symbol('W_rG'),W_rG)) + r" \text{ lbf}"))

<IPython.core.display.Math object>

compute friction force

In [141]:
W_f = mu * W_tG / ( np.cos(var_lambda * np.pi / 180) *np.cos(phi_n * np.pi / 180) - mu * np.sin(var_lambda * np.pi / 180) )
display(Math(latex(sp.Eq(sp.Symbol('W_f'),W_f)) + r" \text{ lbf}"))

<IPython.core.display.Math object>

## Compute Power this is kinda sketchy

Power loss due to friction

$$P_L=\frac{v_sW_f}{33000}$$

In [115]:
P_L = v_s * W_f / 33000
print(f"{P_L=} HP")

P_L=0.020091353084077286 HP


compute output power



In [116]:
if power is None:
    power = T_o * n_g / 63000
power

0.047619047619047616

Compute the input power

In [117]:
##Two different formulas for P_i
P_i = None
if(power is not None):
    P_i = power + P_L
else:
    p_i = -1 * 10**4
    assert(False)
P_i

0.0677104007031249

Compute efficiency $\eta$

In [142]:
eta = power / P_i
display(Math(latex(sp.Eq(sp.Symbol('eta'),eta))))

<IPython.core.display.Math object>

## Find correction Factors

Find lewis form factor, $y$

![Rack and Pinion Image](rack_pinion_images/Table10-5.png)


In [145]:

# Create a dictionary from number to number
lewis_form_dict = {14.5: 0.1, 20: 0.125, 25: 0.15, 30: 0.175}

y = lewis_form_dict.get(phi_n)
print(f"{y=}")



y=0.1


Compute k_v

$$K_v = \frac{1200}{1200+v_{tG}}$$

In [146]:
k_v = 1200 / (1200 + v_tG)
display(Math(latex(sp.Eq(sp.Symbol('k_v'),k_v))))

<IPython.core.display.Math object>

compute dynamic load $W_d$

In [147]:
W_d = W_tG / k_v
display(Math(latex(sp.Eq(sp.Symbol('W_d'),W_d)) + r" \text{ lbf}"))

<IPython.core.display.Math object>

Find stress in gear teeth

In [148]:
sigma = W_d / (y * F * p_n)
display(Math(latex(sp.Eq(sp.Symbol('sigma'),sigma)) + r" \text{ psi}"))

<IPython.core.display.Math object>

### All the following have annoyinh equations / graphs in the bible

Find C_s FROM TABLE

In [124]:
C_s = 1000# None
if C_s is None:
    Warning("C_s is None")
# assert(C_s is not None)

FInd C_m

In [125]:
C_m = None
m_g = VR

if 6 < m_g and m_g < 20:
    C_m = 0.02 * np.sqrt(-1 * m_g**2 + 40*m_g - 76) + 0.46
elif m_g >= 20 and m_g < 76:
    C_m = 0.0107 * np.sqrt(-1 * m_g**2 + 56*m_g + 5146)
elif m_g >= 76:
    C_m = 1.1483 - 0.00658 * m_g

assert(C_m is not None)
C_m

0.8139036429455271

Find C_v

In [126]:
C_v = None
if 0 < v_s and v_s < 700:
    C_v = 0.659 * np.exp(-0.0011 * v_s)
elif v_s > 700  and v_s < 3000:
    C_v = 13.31 * v_s ** (-0.571)
elif v_s > 3000:
    C_s = 65.52 * v_s ** (-0.774)

assert(C_v is not None)
C_v

0.42725219030387684

Find $F_e$

this formula is possibly wrong

In [127]:
F_e = None

assert(False) # this formula is wrong

if F < D_w / 3:
    F_e = F
elif F >= D_w / 3:
    F_e = D_w / 3

assert(F_e is not None)
F_e = F

##

## Find rated tangential load

$$W_{tR}=C_sD_G^{0.8}F_eC_mC_v$$

In [128]:
W_tR = C_s * (D_g ** 0.8) * F_e * C_m * C_v
W_tR

658.848104103243

Check if the design is satisfactory to resist pitting:

$$W_{tR}>W_{tG}$$

In [129]:
assert(W_tR > W_tG)

Need to find F and add pitting formulas